Author: Ryan Timbrook (RTIMBROO)  
DATE: 12/3/2019 <br>
Topic: 

## 1. Objective:
-----------------------------------------------------------------------------------------------------

In [2]:
import pandas as pd
import numpy as np
import json
import os
from os import path
import fnmatch
import io
import re
import string
from datetime import date
from datetime import time
from datetime import datetime

In [3]:
# custome python packages
import rtimbroo_utils as br  

In [4]:
# set global properties
notebook_file_name = 'merge_training_datasets'
report_file_name = 'merge_training_datasets'
app_name = 'merge_training_datasets'
log_level = 10 # 10-DEBUG, 20-INFO, 30-WARNING, 40-ERROR, 50-CRITICAL

# setup working directory structure
# set global properties
dataDir = './data/tables'
logOutDir = './logs'
imageDir = './images'

# create base output directories if they don't exist
if not os.path.exists(logOutDir): os.mkdir(logOutDir)
if not os.path.exists(imageDir): os.mkdir(imageDir)
if not os.path.exists(dataDir): os.mkdir(dataDir)

In [5]:
# get a logger for troubleshooting / data exploration
logger = br.getFileLogger(logOutDir+'/',app_name,level=log_level)

In [31]:
fig8_emo = pd.read_csv(f'{dataDir}/figure8_emotionintext_text_emotion.csv', encoding='latin')
fig8_gop = pd.read_csv(f'{dataDir}/figure8_GOP_REL_ONLY.csv', encoding='latin')
noemo_train_16 = pd.read_csv(f'{dataDir}/training_16_processed_noemoticon.csv', encoding='latin')
train_sent_analysis2 = pd.read_csv(f'{dataDir}/train_kaggle_sentiment_analysis_2.csv', encoding='latin')
airline_sent = pd.read_csv(f'{dataDir}/airline_sent.csv', encoding='latin')

In [19]:
logger.info(f'{fig8_emo.shape}')
fig8_emo.head()

(40000, 4)


,tweet_id,sentiment,author,content
0,1956967341,empty,xoshayzers,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,wannamama,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,coolfunky,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,czareaquino,wants to hang out with friends SOON!
4,1956968416,neutral,xkilljoyx,@dannycastillo We want to trade with someone w...


In [21]:
np.unique(fig8_emo['sentiment'])

array(['anger', 'boredom', 'empty', 'enthusiasm', 'fun', 'happiness',
       'hate', 'love', 'neutral', 'relief', 'sadness', 'surprise',
       'worry'], dtype=object)

In [22]:
logger.info(f'{fig8_gop.shape}')
fig8_gop.head()

(13871, 20)


,candidate,candidate:confidence,relevant_yn,relevant_yn:confidence,sentiment,sentiment:confidence,subject_matter,subject_matter:confidence,candidate_gold,name,relevant_yn_gold,retweet_count,sentiment_gold,subject_matter_gold,text,tweet_coord,tweet_created,tweet_id,tweet_location,user_timezone
0,No candidate mentioned,1.0,yes,1.0,Neutral,0.6578,None of the above,1.0000,NaN,I_Am_Kenzi,NaN,5,NaN,NaN,RT @NancyLeeGrahn: How did everyone feel about...,NaN,8/7/15 9:54,6.296970e+17,NaN,Quito
1,Scott Walker,1.0,yes,1.0,Positive,0.6333,None of the above,1.0000,NaN,PeacefulQuest,NaN,26,NaN,NaN,RT @ScottWalker: Didn't catch the full #GOPdeb...,NaN,8/7/15 9:54,6.296970e+17,NaN,NaN
2,No candidate mentioned,1.0,yes,1.0,Neutral,0.6629,None of the above,0.6629,NaN,PussssyCroook,NaN,27,NaN,NaN,RT @TJMShow: No mention of Tamir Rice and the ...,NaN,8/7/15 9:54,6.296970e+17,NaN,NaN
3,No candidate mentioned,1.0,yes,1.0,Positive,1.0000,None of the above,0.7039,NaN,MattFromTexas31,NaN,138,NaN,NaN,RT @RobGeorge: That Carly Fiorina is trending ...,NaN,8/7/15 9:54,6.296970e+17,Texas,Central Time (US & Canada)
4,Donald Trump,1.0,yes,1.0,Positive,0.7045,None of the above,1.0000,NaN,sharonDay5,NaN,156,NaN,NaN,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,NaN,8/7/15 9:54,6.296970e+17,NaN,Arizona


In [23]:
np.unique(fig8_gop['sentiment'])

array(['Negative', 'Neutral', 'Positive'], dtype=object)

In [25]:
fig8_gop_sub_df = fig8_gop[['tweet_id','sentiment','sentiment:confidence','text']]

In [26]:
fig8_gop_sub_df.head()

,tweet_id,sentiment,sentiment:confidence,text
0,6.296970e+17,Neutral,0.6578,RT @NancyLeeGrahn: How did everyone feel about...
1,6.296970e+17,Positive,0.6333,RT @ScottWalker: Didn't catch the full #GOPdeb...
2,6.296970e+17,Neutral,0.6629,RT @TJMShow: No mention of Tamir Rice and the ...
3,6.296970e+17,Positive,1.0000,RT @RobGeorge: That Carly Fiorina is trending ...
4,6.296970e+17,Positive,0.7045,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...


In [28]:
logger.info(f'{train_sent_analysis2.shape}')
train_sent_analysis2.head()

(99989, 3)


,ItemID,Sentiment,SentimentText
0,1,0,is so sad for my APL frie...
1,2,0,I missed the New Moon trail...
2,3,1,omg its already 7:30 :O
3,4,0,.. Omgaga. Im sooo im gunna CRy. I'...
4,5,0,i think mi bf is cheating on me!!! ...


In [29]:
np.unique(train_sent_analysis2['Sentiment'])

array([0, 1], dtype=int64)

In [32]:
logger.info(f'{airline_sent.shape}')
airline_sent.head()

(14640, 15)


,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [33]:
airline_sent_sub_df = airline_sent[['tweet_id','airline_sentiment','airline_sentiment_confidence','text']]

In [34]:
airline_sent_sub_df.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,text
0,570306133677760513,neutral,1.0000,@VirginAmerica What @dhepburn said.
1,570301130888122368,positive,0.3486,@VirginAmerica plus you've added commercials t...
2,570301083672813571,neutral,0.6837,@VirginAmerica I didn't today... Must mean I n...
3,570301031407624196,negative,1.0000,@VirginAmerica it's really aggressive to blast...
4,570300817074462722,negative,1.0000,@VirginAmerica and it's a really big bad thing...


In [35]:
# data sets to keep and merge
# airline_sent, fig8_gop_sub_df

airline_sent_sub_df.columns = ['id','sentiment','sentiment_confidence','text']
fig8_gop_sub_df.columns = ['id','sentiment','sentiment_confidence','text']

In [39]:
df_labels = []
for i in range(0,airline_sent_sub_df.shape[0]):
    df_labels.append('airline')

airline_sent_sub_df['df_label'] = df_labels
airline_sent_sub_df.head()

C:\Users\rt310\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


,id,sentiment,sentiment_confidence,text,df_label
0,570306133677760513,neutral,1.0000,@VirginAmerica What @dhepburn said.,airline
1,570301130888122368,positive,0.3486,@VirginAmerica plus you've added commercials t...,airline
2,570301083672813571,neutral,0.6837,@VirginAmerica I didn't today... Must mean I n...,airline
3,570301031407624196,negative,1.0000,@VirginAmerica it's really aggressive to blast...,airline
4,570300817074462722,negative,1.0000,@VirginAmerica and it's a really big bad thing...,airline


In [40]:
df_labels = []
for i in range(0,fig8_gop_sub_df.shape[0]):
    df_labels.append('gop')

fig8_gop_sub_df['df_label'] = df_labels

fig8_gop_sub_df['sentiment'] = fig8_gop_sub_df['sentiment'].str.lower()
fig8_gop_sub_df.head()

,id,sentiment,sentiment_confidence,text,df_label
0,6.296970e+17,neutral,0.6578,RT @NancyLeeGrahn: How did everyone feel about...,gop
1,6.296970e+17,positive,0.6333,RT @ScottWalker: Didn't catch the full #GOPdeb...,gop
2,6.296970e+17,neutral,0.6629,RT @TJMShow: No mention of Tamir Rice and the ...,gop
3,6.296970e+17,positive,1.0000,RT @RobGeorge: That Carly Fiorina is trending ...,gop
4,6.296970e+17,positive,0.7045,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,gop


In [41]:
train_sentiment_master_df = pd.DataFrame()


train_sentiment_master_df = train_sentiment_master_df.append(airline_sent_sub_df, ignore_index=True)
train_sentiment_master_df = train_sentiment_master_df.append(fig8_gop_sub_df, ignore_index=True)

In [42]:
train_sentiment_master_df.shape

(28511, 5)

In [43]:
train_sentiment_master_df.head()

,id,sentiment,sentiment_confidence,text,df_label
0,5.703061e+17,neutral,1.0000,@VirginAmerica What @dhepburn said.,airline
1,5.703011e+17,positive,0.3486,@VirginAmerica plus you've added commercials t...,airline
2,5.703011e+17,neutral,0.6837,@VirginAmerica I didn't today... Must mean I n...,airline
3,5.703010e+17,negative,1.0000,@VirginAmerica it's really aggressive to blast...,airline
4,5.703008e+17,negative,1.0000,@VirginAmerica and it's a really big bad thing...,airline


In [45]:
train_sentiment_master_df.to_csv(f'{dataDir}/train_tweets_master.csv', index=False)